compare between model results and survey data: see how the results change/improve when we use coefficients estimated from our survey
- data: persons
- variables: distance_to_work, distance_to_school, is_worker, is_grade/highschool/university
- distance bins from [workplace_location.csv](https://github.com/psrc/psrc_activitysim/blob/main/configs_dev/workplace_location.csv)



In [1]:
import os
import pandas as pd
import numpy as np
import pyodbc
from pathlib import Path
import plotly.express as px
from plotly.subplots import make_subplots

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# path
## person data
p_model_survey_perdata = "R:/e2projects_two/activitysim/estimation/2017_2019_data/validation_data/survey_outputs/final_persons.csv"
p_psrc_model_perdata = "R:/e2projects_two/activitysim/estimation/2017_2019_data/validation_data/model_outputs/pipeline.parquetpipeline/persons/mp_households.parquet"

## Person Data

In [2]:
# read data

# model data
per_data_model = pd.read_parquet(p_psrc_model_perdata).reset_index()
# add weight to model data with all 1
per_data_model['hh_weight_2017_2019'] = np.repeat(1, len(per_data_model))


# survey data
per_data_survey = pd.read_csv(p_model_survey_perdata).groupby('person_id_elmer').first().reset_index() # remove duplicates
# unweighted survey data
per_data_survey_unweighted = per_data_survey.copy()
per_data_survey_unweighted['hh_weight_2017_2019'] = 1

print(f"person counts \n"
      f"- model results: {len(per_data_model)}\n"
      f"- survey results: {per_data_survey['hh_weight_2017_2019'].sum()}\n"
      f"- unweighted survey: {per_data_survey_unweighted['hh_weight_2017_2019'].sum()}\n"
      # f"group dividers:\n"
      # f"{var_group}"
      )

person counts 
- model results: 4053154
- survey results: 3190247.7283157064
- unweighted survey: 10169



In [3]:
#| warning: false

# process model and survey data for summary
def data_process(df: pd.DataFrame, data_source: str) -> pd.DataFrame:

    # add data source
    df['source'] = data_source
    # distance to school bins from workplace_location.csv
    df['distance_to_work_bin'] = pd.cut(df['distance_to_work'], bins=[0,1,2,5,15,9999],
                                          labels=['util_dist_0_1', 'util_dist_1_2', 'util_dist_2_5',
                                                  'util_dist_5_15', 'util_dist_15_up'])
    # Create bins: bins of 2 miles up to 60 miles
    max_bin = 60
    bin_size = 2
    df['d_work_bin_60mi'] = pd.cut(df['distance_to_work'], bins=np.arange(0, max_bin+bin_size, bin_size), labels=[str(i) for i in np.arange(0, max_bin, bin_size)])

    return df

# match columns and concat all source into hh_data
col_list = ['person_id', 'household_id', 'hh_weight_2017_2019',
            'is_worker','distance_to_work']

# combine both sets of data
per_data = pd.concat([data_process(per_data_model[col_list], "model results").copy(),
                      data_process(per_data_survey[col_list], "survey data").copy(),
                      data_process(per_data_survey_unweighted[col_list], "unweighted survey data").copy()])


# per_data

C:\Users\Modeller\AppData\Local\Temp\ipykernel_6284\1405035287.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Modeller\AppData\Local\Temp\ipykernel_6284\1405035287.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\Modeller\AppData\Local\Temp\ipykernel_6284\1405035287.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Modeller\AppData\Local\Temp\ipykernel_6284\1405035287.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Modeller\AppData\Local\Temp\ipykernel_6284\1405035287.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

## Distance to work

In [4]:
def plot_distance(df:pd.DataFrame, group:str, title_name:str):
    count = df.loc[(df['source']=='model results') & (df[group]),['distance_to_work_bin']].value_counts()
    print(f"model person count =\n"
          f"{count.sort_values()}")

    # plot1
    df_plot = df.loc[(df[group]) & (df['distance_to_work_bin'] != float('nan'))].groupby(['source','distance_to_work_bin'])['hh_weight_2017_2019'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['hh_weight_2017_2019'].\
            apply(lambda x: 100 * x / float(x.sum()))
    # df_plot
    #
    fig1 = px.bar(df_plot, x='distance_to_work_bin', y="percentage", color="source", barmode="group",template="simple_white",
                  title=title_name)
    fig1.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig1.update_layout(height=400, width=700, font=dict(size=11))
    fig1.show()

    # plot2
    df_plot = per_data.loc[(per_data[group]) & (per_data['d_work_bin_60mi'].notna())].groupby(['source','d_work_bin_60mi'])['hh_weight_2017_2019'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['hh_weight_2017_2019'].\
                apply(lambda x: 100 * x / float(x.sum()))
    # df_plot
    #
    fig2 = px.line(df_plot, x='d_work_bin_60mi', y="percentage", color="source", template="simple_white",
                   title=title_name)
    fig2.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig2.update_layout(height=400, width=700, font=dict(size=11))
    fig2.show()

# distance to work
plot_distance(per_data,'is_worker',"worker: distance to work")

model person count =
distance_to_work_bin
util_dist_0_1            66909
util_dist_1_2           123074
util_dist_2_5           282761
util_dist_15_up         554785
util_dist_5_15          675510
dtype: int64
